In [ ]:
!pip install transformers sentencepiece torch datasets

In [ ]:
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric
import torch

In [ ]:
# Load pre-trained model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-fr"  # English to French
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [ ]:
def translate_sentence(sentence):
    """
    Translates a given sentence using the pre-trained model.
    """
    inputs = tokenizer(sentence, return_tensors="pt")
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Example Usage
sentence = "Hello, how are you?"
print(f"Original: {sentence}")
print(f"Translated: {translate_sentence(sentence)}")

In [ ]:
# Load a dataset for training (Replace 'your_dataset.csv' with the actual dataset file)
dataset = load_dataset('csv', data_files={'train': 'your_dataset.csv'})
print(dataset)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_total_limit=2,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
)

# Train the model
trainer.train()

In [ ]:
# Example Evaluation using BLEU score
test_sentences = [
    "This is a test sentence.",
    "Another sentence for translation.",
    "How is the weather today?",
]

references = [
    ["Ceci est une phrase de test."],
    ["Une autre phrase pour la traduction."],
    ["Quel temps fait-il aujourd'hui ?"],
]

predictions = [translate_sentence(sentence) for sentence in test_sentences]
metric = load_metric("bleu")
results = metric.compute(predictions=[[pred] for pred in predictions], references=references)
print(f"BLEU score: {results['bleu']}")